In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.special

In [54]:
inverse_softmax(norm_alphas[0][0], C), F.softmax(inverse_softmax(norm_alphas[0][0], C), dim=-1)

(tensor([ 0.0008,  0.0017, -0.0001,  0.0014, -0.0016, -0.0014,  0.0006],
        grad_fn=<AddBackward0>),
 tensor([0.1429, 0.1431, 0.1428, 0.1430, 0.1426, 0.1426, 0.1429],
        grad_fn=<SoftmaxBackward>))

In [459]:
n_ops = 7
n_nodes = 3

alphas = []
norm_alphas = []


for i in range(n_nodes):
    a = nn.Parameter(
        1e-3 * torch.randn(i + 2, n_ops))
    alphas.append(a)
    norm_alphas.append(F.softmax(a, dim=-1))


def inverse_softmax(x, C):
    return torch.log(x) + C

def calc_C_term(a):
#     e_xi = torch.exp(a)
#     sum_i = torch.sum(e_xi)
#     c_i = torch.log(sum_i)
#     return c_i
    return torch.log(torch.tensor([10.]))[0]


# Fully, allows increase 
def increase_op(row_idx, edge_idx, op_idx, prob=0.3, n_ops=7):
    C = calc_C_term(alphas[row_idx][edge_idx])
    
    # Set short-hands
#     curr_op = norm_alphas[row_idx][edge_idx][op_idx]
#     curr_edge = norm_alphas[row_idx][edge_idx]
    curr_op = torch.tensor([0.32, 0.05, 0.01, 0.03, 0.23, 0.36, 0.00])[op_idx]
    curr_edge = torch.tensor([0.32, 0.05, 0.01, 0.03, 0.23, 0.36, 0.00])
    
    
    # Allow for increasing to 0.99
    if curr_op + prob > 1.0:
        surplus = curr_op + prob - 0.99
        prob -= surplus
    
    if curr_op + prob < 1.0:
        # Increase chosen op
        with torch.no_grad():
            curr_op += prob

        # Align the other probabilities by lowering them
#         indices = np.delete(np.arange(n_ops), op_idx)
        with torch.no_grad():
            curr_edge += 0.01
    
    with torch.no_grad():
        alphas[row_idx][edge_idx] = inverse_softmax(curr_edge, C)
        
        
    # Display for testing
    print("orig", alphas[row_idx][edge_idx])
    norm_alphas[row_idx][edge_idx] = F.softmax(alphas[row_idx][edge_idx], dim=-1)
    print("norm", norm_alphas[row_idx][edge_idx], torch.sum(norm_alphas[row_idx][edge_idx]))
    
    
    
def decrease_op(row_idx, edge_idx, op_idx, prob=0.3, n_ops=7):
    C = calc_C_term(alphas[row_idx][edge_idx])
    
    # Set short-hands
    curr_op = norm_alphas[row_idx][edge_idx][op_idx]
    curr_edge = norm_alphas[row_idx][edge_idx]
    
    # Allow for increasing to 0.99
    if curr_op - prob < 0.0:
        surplus = prob - curr_op + 0.01
        print(surplus)
        prob -= surplus
        print(prob)
    
    if curr_op - prob > 0.0:
        # Increase chosen op
        with torch.no_grad():
            curr_op -= prob

        # Align the other probabilities by lowering them
        indices = np.arange(n_ops), op_idx
        with torch.no_grad():
            norm_alphas[row_idx][edge_idx] += 0.01
    
    with torch.no_grad():
        alphas[row_idx][edge_idx] = inverse_softmax(curr_edge, C)
        
        
    # Display for testing
    print("orig", alphas[row_idx][edge_idx])
    norm_alphas[row_idx][edge_idx] = F.softmax(alphas[row_idx][edge_idx], dim=-1)
    print("norm", norm_alphas[row_idx][edge_idx], torch.sum(norm_alphas[row_idx][edge_idx]))
    
increase_op(0, 0, 0)

orig tensor([ 1.1632, -0.5108, -1.6094, -0.9163,  0.8755,  1.3083, -2.3026],
       grad_fn=<SelectBackward>)
norm tensor([0.3019, 0.0566, 0.0189, 0.0377, 0.2264, 0.3491, 0.0094],
       grad_fn=<SelectBackward>) tensor(1.0000, grad_fn=<SumBackward0>)
